## Aprendizado de máquina (*Machine Learning*)

De forma simples, o aprendizado de máquina consiste em obter **dados** e detectar **padrões** nesses dados

Assim, podemos utilizar esses padrões para **agrupar** conjuntos de dados similares ou criar um **modelo** que consiga **prever** um resultado (nosso **objetivo**) com base em novos dados. 

Vemos que a definição de vários conceitos é necessária. Podemos detalhar melhor cada uma dessas etapas com um exemplo. Como discutimos em outros Notebooks sobre quiminformática (uso de métodos computacionais para resolver problemas em química), podemos usar como exemplo o seguinte problema: 

**Problema: Desejamos criar um modelo que nos permita prever se um composto químico será solúvel em água.**

Esse é um problema relevante para diversas áreas da química. Imagine que um pesquisador esteja buscando desenvolver um fármaco para ser tomado por via oral. Seria excelente se ele pudesse, por exemplo, prever a solubilidade de um composto antes de sintetizá-lo, pois compostos devem apresentar solubilidade em água razoável para serem administrados por via oral.

Como poderíamos prosseguir para criar esse modelo?

Uma forma de resolver esse problema seria observar a solubilidade de diferentes compostos e tentar desenvolver uma **teoria** geral, baseada nas propriedades fundamentais das substâncias químicas, que descrevesse, através de uma função matemática, o valor da solubilidade da substância. Pode parecer exagerado, mas somos capazes de realizar isso para diversas propriedades, como cálculo de energia de orbitais atômicos e moleculares (ainda que de forma aproximada, e não exata).

Todavia, uma teoria geral para a solubilidade de compostos químicos (ainda) está além do nosso alcance, pois existem muitos fatores envolvidos. O que fazer nesse caso? Conforme descrito anteriormente, podemos tentar a abordagem do aprendizado de máquina. Para isso, devemos começar coletando um número grande de dados (em geral, quanto mais, melhor). 

O que são os dados? Dados consistem em diversos exemplos daquilo que estamos tentando prever. No nosso caso, os dados seriam o valor numérico da solubilidade de compostos químicos, e suas estruturas químicas. 

Como obter dados? Os dados precisam ser gerados, porém, uma vez gerados, podem ser armazenados em **bancos de dados**. Dessa forma, em vez de medir a solubilidade de diversos compostos químicos, podemos simplesmente acessar um banco de dados contendo essa informação. Um banco de dados pode ser algo tão simples quanto uma tabela contendo, em uma coluna, o nome (ou a estrutura química) de uma substância, e o valor numérico de sua solubilidade em água (por exemplo: 1,2 mg/L) em outra coluna. [Este link](https://www.kaggle.com/c/drug-solubility-challenge/data) contém um exemplo de um banco de dados público com valores de solubilidade de compostos químicos. O banco de dados pode ser baixado, e parte dele é mostrada a seguir:


In [1]:
import pandas as pd

df = pd.read_csv("../datasets/delaney-processed.csv")

df[['Compound ID', 'measured log solubility in mols per litre']].head()

,Compound ID,measured log solubility in mols per litre
0,Amigdalin,-0.77
1,Fenfuram,-3.30
2,citral,-2.06
3,Picene,-7.87
4,Thiophene,-1.33


Aqui, duas colunas correspondentes aos cinco primeiros exemplos da tabela são mostradas. Veja que esse banco de dados contém, dentre outras colunas não mostradas, identificadores para as substâncias químicas ('Compound ID') e sua solubilidade em água (em mols por litro).

Seria isso suficiente para gerarmos um modelo preditivo? A resposta é "não", pois, para prever uma propriedade, é necessário extrair informações de outras propriedades. Por exemplo, é impossível (de forma não-clarividente) prever se um time de futebol vai ganhar ou perder uma partida ou mesmo se vai chover amanhã. Porém, se utilizarmos o histórico das partidas dos dois times, ou medições climáticas (direção do vento, umidade, chegada de uma frente fria), podemos extrair informações e fazer predições com razoável precisão de tais eventos futuros. Da mesma forma, precisamos de propriedades que descrevam nossos dados de forma mais completa possível, e que nos permitam criar um modelo. A essas propriedades damos o nome de **descritores**.

Como podemos descrever nossos dados? Precisamos de propriedades que carreguem informação relevante. Por exemplo, para prever se uma pessoa comprará fraldas ao ir ao mercado, um bom descritor seria: *essa pessoa tem filhos recém-nascidos?* e um péssimo descritor seria a cor dos sapatos da pessoa (isso não deve ter qualquer influência na compra de fraldas).

Vemos que no banco de dados de solubilidade (agora mostrando todas as colunas), alguns descritores nos são fornecidos:


In [2]:
df.head()

,Compound ID,ESOL predicted log solubility in mols per litre,Minimum Degree,Molecular Weight,Number of H-Bond Donors,Number of Rings,Number of Rotatable Bonds,Polar Surface Area,measured log solubility in mols per litre,smiles
0,Amigdalin,-0.974,1,457.432,7,3,7,202.32,-0.77,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...
1,Fenfuram,-2.885,1,201.225,1,2,2,42.24,-3.30,Cc1occc1C(=O)Nc2ccccc2
2,citral,-2.579,1,152.237,0,0,4,17.07,-2.06,CC(C)=CCCC(C)=CC(=O)
3,Picene,-6.618,2,278.354,0,5,0,0.00,-7.87,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43
4,Thiophene,-2.232,2,84.143,0,1,0,0.00,-1.33,c1ccsc1


In [3]:
df.columns

Index(['Compound ID', 'ESOL predicted log solubility in mols per litre',
       'Minimum Degree', 'Molecular Weight', 'Number of H-Bond Donors',
       'Number of Rings', 'Number of Rotatable Bonds', 'Polar Surface Area',
       'measured log solubility in mols per litre', 'smiles'],
      dtype='object')

Vejamos o significado dessas colunas:
- 'Compound ID': identificador da molécula
- 'ESOL predicted log solubility in mols per litre': valor de solubilidade predito pelo modelo ESOL
- 'Minimum Degree': não conheço esse descritor, mas parece estar relacionado à saturação da molécula
- 'Molecular Weight': peso molecular, ou massa molecular relativa. Essa é uma propriedade molecular muito usado em química
- 'Number of H-Bond Donors': número de átomos doadores de ligação de hidrogênio (no caso, N e O)
- 'Number of Rings': número de anéis (sistemas cíclicos)
- 'Number of Rotatable Bonds': número de ligações rotacionáveis (ou seja, ligações simples com livre rotação)
- 'Polar Surface Area': área polar superficial
- 'measured log solubility in mols per litre': solubilidade medida
- 'smiles': SMILES, uma forma de representação molecular muito usada em quiminformática


Que valores poderíamos utilizar para treinar nosso modelo de aprendizado de máquina?

Lembre-se de que para treinar um modelo precisamos ter um alvo bem definido (o que queremos prever) e dados que contêm informação que podem ser utilizadas para prever o alvo (descritores). Para facilitar, podemos chamar a variável alvo de `y` e os descritores como `X`.

Com esse conjunto de dados poderíamos, por exemplo, utilizar os valores de peso molecular, número de átomos doadores de ligação de hidrogênio, número de anéis, número de ligações rotacionáveis e área polar superficial para treinar nosso modelo.

Não seria possível usar a coluna 'Compound ID', pois um modelo não saberia como extrair informação do nome de um composto.

A coluna 'smiles' é mais interessante: ela também contém texto, mas cada SMILES representa a estrutura molecular de um composto químico. A partir dos SMILES, podemos gerar todos os descritores apresentados, e muitos outros. Existem até modelos mais avançados que aceitam SMILES como entrada (a arquitetura do modelo é capaz de extrair a informação estrutural diretamente do texto do SMILES). Mas, para os fins desse Notebook, vamos nos restringir aos descritores apresentados.

## Como treinar seu modelo?

Vamos começar definindo nosso descritores e nosso alvo, selecionando as colunas apropriadas (veja o Notebook sobre o módulo Pandas caso não entenda o código a seguir):

In [4]:
import pandas as pd
X = df[['Molecular Weight', 'Number of H-Bond Donors',
       'Number of Rings', 'Number of Rotatable Bonds', 
       'Polar Surface Area']]
y = df['measured log solubility in mols per litre'].ravel()

Agora, treinar um modelo é simples:

In [5]:
from sklearn.ensemble import RandomForestRegressor

modelo = RandomForestRegressor().fit(X, y)

E pronto! A variável `modelo` contém nosso primeiro modelo de aprendizado de máquina.

Muitas coisas aconteceram nessas poucas linhas de código:

- importamos o algoritmo que treina um modelo do tipo *Random Forest Regressor* (algoritmo = *Random forest*, tipo do problema = regressão) da biblioteca *scikit-learn*
- passamos os valores dos descritores (X) e do alvo (y) ao algoritmo
- diversos modelos do tipo "árvores de decisão" foram treinados em diferentes subconjuntos de X e y (é assim que o *Random Forest* cria o modelo. Não vou entrar em detalhes, mas uma excelente fonte para começar a entender esses modelos são: o canal do YouTube [Statquest](https://www.youtube.com/watch?v=J4Wdy0Wc_xQ) e o livro [An Introduction to Statistical Learning](https://www.statlearning.com/))

Podemos usar nosso modelo, contido na variável `modelo`, para prever a solubilidade de qualquer composto (com base nos cinco descritores utilizados). Como exemplo, vamos ver qual valor o modelo prevê para o primeiro composto da tabela:

In [6]:
y_pred = modelo.predict(pd.DataFrame(X.loc[0]).T)

print("Valor predito pelo modelo =", y_pred)
print("Valor real ="               , y[0])

Valor predito pelo modelo = [-1.15719]
Valor real = -0.77


Essa foi uma predição boa? Como é o desempenho do modelo para outras amostras? Para responder a essas questões, será necessário utilizar conceitos como **métricas de qualidade**. Mas isso é material para outros Notebooks...

Conforme for avançando nos estudos, perceberá que cometi diversos erros/imprecisões nesse tutorial. Por exemplo:
- não separei um conjunto de treinamento e um de teste
- usei poucos descritores
- avaliei a resposta do modelo em uma amostra usada para treinar o modelo
- não normalizei os dados (será que isso é um erro mesmo?)

Tente entender por que (ou em que contextos) esse tipo de escolha pode prejudicar os modelos de aprendizado de máquina.

Como comentário extra, note que o aprendizado de máquina difere do desenvolvimento de *software* tradicional, pois não escrevemos diretamente as linhas de comando que determinarão a saída do modelo. Por exemplo, se fôssemos usar nosso conhecimento de química para escrever um modelo preditivo de solubilidade, poderíamos pensar em algo assim:

```
Comece com um valor-base de solubilidade igual a 100 mg por litro.

     - Se a massa molecular for menor que 300: aumente o valor de solubilidade em 20 unidades
     - Se a massa molecular estiver entre 300 e 500: aumente o valor de solubilidade em 10 unidades
     - Se a massa molecular estiver entre 500 e 700: diminua o valor de solubilidade em 10 unidades
     - Se a massa molecular for maior que 700: diminua o valor de solubilidade em 20 unidades


```

E assim por diante, para cada propriedade.

Todavia, usando aprendizado de máquina, deixamos que "os dados falem por si só", ou seja, o algoritmo tentará várias possibilidades até encontrar a melhor descrição para nosso problema.


### Conclusão

Para fechar, reveja os conceitos apresentados e compare-os com o exemplo mostrado. Tente identificar o que são:
- Dados
- Descritores
- Variável Alvo
- Algoritmo
- Modelo
- Predição
